In [1]:
import pandas as pd
import numpy as np
import sys
import re
import gc

!pip install -U -q PyDrive
 
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

from google.colab import drive
drive.mount('/content/drive')

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
file_list = drive.ListFile({'q': "'1IMT5wkteYw7oVhN5exjoHmFtGaWSVu28' in parents and trashed=false"}).GetList()
for file1 in file_list:
  print('title: %s, id: %s' % (file1['title'], file1['id']))

title: IRAhandle_tweets_8.csv, id: 1tncgM0Qk8tVwnlDpcS6ExcKLgXSQFivr
title: IRAhandle_tweets_6.csv, id: 1Zj7y_-3A-7Fos-zGCHEKgPBoQvjmysCS
title: IRAhandle_tweets_7.csv, id: 1jI5nQyznspfgk6x1RqJeYs184R8rDAH3
title: IRAhandle_tweets_9.csv, id: 14fs5-wJUPRJ5_n_KNWOFkXOF_JgQBcvn
title: IRAhandle_tweets_5.csv, id: 1zqwycOdInUlWqs6n1cg1O9ahs-LdfDH-
title: IRAhandle_tweets_3.csv, id: 1Rk7XNjnBzeCPi2o-TV1pD8qbMFOxrnXH
title: IRAhandle_tweets_4.csv, id: 1QSArmXCXdkfWHlcTVqPwnzdAjjkOenF7
title: IRAhandle_tweets_2.csv, id: 1bNSGGniOObCfzTZ085FyFVjqNpC5cRnD
title: IRAhandle_tweets_1.csv, id: 1TEkomSxdkUuhkpWTvB1YNZFsvEVP_Pwk


In [0]:
trolldf = pd.DataFrame()
for file in file_list:
    downloaded = drive.CreateFile({'id': file['id']})
    downloaded.GetContentFile(file['title'])
    df_acc = pd.read_csv(file['title'])
    trolldf = trolldf.append(df_acc, ignore_index = True, sort = False)
#trolldf.head()

In [4]:
trolldf.head()

,external_author_id,author,content,region,language,publish_date,harvested_date,following,followers,updates,post_type,account_type,new_june_2018,retweet,account_category
0,1.651693e+09,STAYWOKE88,#Illinois Dairy #Queen Franchisee Loses Store ...,United States,English,1/10/2017 0:43,1/10/2017 0:44,549,465,3574,RETWEET,left,0,1,LeftTroll
1,1.651693e+09,STAYWOKE88,#NoEthicsNoHearing https://t.co/q8suwAO2sB,United States,English,1/10/2017 0:44,1/10/2017 0:44,549,465,3575,RETWEET,left,0,1,LeftTroll
2,1.651693e+09,STAYWOKE88,Progressive narratives about what’s driving ma...,United States,English,1/10/2017 12:50,1/10/2017 12:51,549,466,3594,RETWEET,left,0,1,LeftTroll
3,1.651693e+09,STAYWOKE88,Finally found his twin. Mitch McConnell https:...,United States,English,1/10/2017 12:50,1/10/2017 12:50,549,466,3592,RETWEET,left,0,1,LeftTroll
4,1.651693e+09,STAYWOKE88,"#DonnieDipshit: ""Confirmations r going great""....",United States,English,1/10/2017 12:50,1/10/2017 12:51,549,466,3593,RETWEET,left,0,1,LeftTroll


In [10]:
trolldf = trolldf[['author', 'following', 'followers']]
trolldf.head()

,author,following,followers
0,STAYWOKE88,549,465
1,STAYWOKE88,549,465
2,STAYWOKE88,549,466
3,STAYWOKE88,549,466
4,STAYWOKE88,549,466


In [5]:
file_list = drive.ListFile({'q': "'1sFcNq_jqE8-vdmC1TlXrh2plS8qy84PZ' in parents and trashed=false"}).GetList()
for file1 in file_list:
  print('title: %s, id: %s' % (file1['title'], file1['id']))

title: nov29.csv, id: 12NSIxIbgiQd8N3i49lcK4DeBpaV6hCWc
title: nov28.csv, id: 1huayYTurbpgpsVI99IoKVyUBkdKGhz_N
title: nov27.csv, id: 1AAAUuIdyq1TjZgO54Njl3s4E4hDzeTAK
title: nov30.csv, id: 1cmPemTfEF61B2TWpwO1f8h9WQxIdxKf8


In [0]:
normaldf = pd.DataFrame()
for file in file_list:
    downloaded = drive.CreateFile({'id': file['id']})
    downloaded.GetContentFile(file['title'])
    df_acc = pd.read_csv(file['title'], usecols = ['author_id','following', 'followers'])
    normaldf = normaldf.append(df_acc, ignore_index = True, sort = False)
#normaldf.head()

In [7]:
normaldf.head()

,author_id,following,followers
0,225551004,244,209
1,184621766,66,143
2,743166519157227520,134,643
3,53706363,192,19651
4,2418453842,306,323


In [11]:
trolldf['troll'] = pd.Series([1] * len(trolldf))
trolldf.head()

,author,following,followers,troll
0,STAYWOKE88,549,465,1
1,STAYWOKE88,549,465,1
2,STAYWOKE88,549,466,1
3,STAYWOKE88,549,466,1
4,STAYWOKE88,549,466,1


In [13]:
normaldf['troll'] = pd.Series([0] * len(normaldf))

normaldf.columns = trolldf.columns

normaldf.head()

,author,following,followers,troll
0,225551004,244,209,0
1,184621766,66,143,0
2,743166519157227520,134,643,0
3,53706363,192,19651,0
4,2418453842,306,323,0


In [0]:
per_profile1 = trolldf.groupby(['author'])



followers_per_profile = per_profile1['followers'].agg(np.amax)
following_per_profile = per_profile1['following'].agg(np.amax)
ratio_followers_following = (followers_per_profile + 1) / (following_per_profile + 1) # smoothing




In [0]:
array2 = [followers_per_profile, following_per_profile, ratio_followers_following]
profile_features = ['Followers', 'Following', 'Ratio_Followers_Following']
df_profile_based1 = pd.DataFrame(columns = profile_features)
feature_columns = zip(profile_features, array2)
for feature, column in feature_columns:
    df_profile_based1[feature] = column
  
troll = [1] * len(df_profile_based1)

df_profile_based1['troll'] = troll

In [0]:
per_profile1 = normaldf.groupby(['author'])



followers_per_profile = per_profile1['followers'].agg(np.amax)
following_per_profile = per_profile1['following'].agg(np.amax)
ratio_followers_following = (followers_per_profile + 1) / (following_per_profile + 1) # smoothing





In [0]:
array2 = [followers_per_profile, following_per_profile, ratio_followers_following]
profile_features = ['Followers', 'Following', 'Ratio_Followers_Following']
df_profile_based2 = pd.DataFrame(columns = profile_features)
feature_columns = zip(profile_features, array2)
for feature, column in feature_columns:
    df_profile_based2[feature] = column

troll = [0] * len(df_profile_based2)

df_profile_based2['troll'] = troll

In [0]:
df_profile_based = df_profile_based2.append(df_profile_based1)

In [19]:
df_profile_based.head()

,Followers,Following,Ratio_Followers_Following,troll
author,,,,
12,3925173,2270,1728.390137,0
57,18554,993,18.667002,0
59,5325,1021,5.211350,0
221,1450,947,1.530591,0
291,1173377,369,3171.291892,0


In [0]:
df_profile_based = df_profile_based.dropna()

In [0]:
from sklearn.model_selection import train_test_split
#df_profile_based.columns
X = df_profile_based.loc[:, df_profile_based.columns[0:3]]

y = df_profile_based.iloc[:, 3]
#X.head()
#y.head()


In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [0]:
from sklearn.linear_model import LogisticRegression
Log_clf = LogisticRegression(solver = 'lbfgs').fit(X_train, y_train)

In [0]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
predicted = Log_clf.predict(X_test)
print('Accuracy : {:.2f}', format(accuracy_score(y_test, predicted)))
print('Precision : {:.2f}', format(precision_score(y_test, predicted)))
print('Recall : {:.2f}', format(recall_score(y_test, predicted)))
print('f1-score : {:.2f}', format(f1_score(y_test, predicted)))


Accuracy : {:.2f} 0.9994933765710541
Precision : {:.2f} 0.0
Recall : {:.2f} 0.0
f1-score : {:.2f} 0.0


In [0]:
from sklearn.tree import DecisionTreeClassifier
Dt_clf = DecisionTreeClassifier().fit(X_train, y_train)

In [0]:
predicted = Dt_clf.predict(X_test)
print('Accuracy : {:.2f}', format(accuracy_score(y_test, predicted)))
print('Precision : {:.2f}', format(precision_score(y_test, predicted)))
print('Recall : {:.2f}', format(recall_score(y_test, predicted)))
print('f1-score : {:.2f}', format(f1_score(y_test, predicted)))

Accuracy : {:.2f} 0.9993940278813042
Precision : {:.2f} 0.012690355329949238
Recall : {:.2f} 0.006944444444444444
f1-score : {:.2f} 0.008976660682226212


In [0]:
from sklearn.ensemble import GradientBoostingClassifier
GBDT_clf = GradientBoostingClassifier().fit(X_train, y_train)

In [0]:
predicted = GBDT_clf.predict(X_test)
print('Accuracy : {:.2f}', format(accuracy_score(y_test, predicted)))
print('Precision : {:.2f}', format(precision_score(y_test, predicted)))
print('Recall : {:.2f}', format(recall_score(y_test, predicted)))
print('f1-score : {:.2f}', format(f1_score(y_test, predicted)))

Accuracy : {:.2f} 0.9995905297096493
Precision : {:.2f} 0.0
Recall : {:.2f} 0.0
f1-score : {:.2f} 0.0
